In [628]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/capstone-testdata/Test_Data.csv
/kaggle/input/temporary/Train_Data.csv


In [629]:
df=pd.read_csv('/kaggle/input/temporary/Train_Data.csv')

In [630]:
avg_values = {
    'ld': df['ld'].mean(),
    'm0': df['m0'].mean(),
    'm1': df['m1'].mean(),
    'm2': df['m2'].mean(),
    'm3': df['m3'].mean(),
    'm4': df['m4'].mean(),
    'm5': df['m5'].mean(),
    'm6': df['m6'].mean(),
    'm7': df['m7'].mean(),
    'm8': df['m8'].mean(),
    'm9': df['m9'].mean(),
    'm10': df['m10'].mean(),
    'm11': df['m11'].mean(),
    'm12': df['m12'].mean(),
    'm13': df['m13'].mean(),
    'm14': df['m14'].mean(),


}



df.fillna(value=avg_values, inplace=True)

In [631]:
df[['trash','ma']] = df['ma'].str.split('m', expand=True)
df=df.drop(['trash'],axis=1)
df['ma'] = df['ma'].astype(int)


def replace_with_ascii(text):
    return ''.join(str(ord(char)) for char in text)
df['pc'] = df['pc'].apply(replace_with_ascii)
df['pc'] = df['pc'].astype(int)

df.dropna(subset=['pred'], inplace=True)

In [632]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [633]:
# Assuming X contains the predictor variables and y contains the target variable
X = df.iloc[:, 0:18]  # Select the columns representing the predictor variables
y = df.iloc[:,18]  # Select the column representing the target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)


In [634]:
test=pd.read_csv("/kaggle/input/capstone-testdata/Test_Data.csv")

In [635]:
avg_values_test = {
    'ld': test['ld'].mean(),
    'm0': test['m0'].mean(),
    'm1': test['m1'].mean(),
    'm2': test['m2'].mean(),
    'm3': test['m3'].mean(),
    'm4': test['m4'].mean(),
    'm5': test['m5'].mean(),
    'm6': test['m6'].mean(),
    'm7': test['m7'].mean(),
    'm8': test['m8'].mean(),
    'm9': test['m9'].mean(),
    'm10': test['m10'].mean(),
    'm11': test['m11'].mean(),
    'm12': test['m12'].mean(),
    'm13': test['m13'].mean(),
    'm14': test['m14'].mean(),


}


test.fillna(value=avg_values_test, inplace=True)

In [636]:
test[['trash','ma']] = test['ma'].str.split('m', expand=True)
test=test.drop(['trash'],axis=1)
test['ma'] = test['ma'].astype(int)


test['pc'] = test['pc'].apply(replace_with_ascii)
test['pc'] = test['pc'].astype(int)

test=test.drop(['index'],axis=1)

In [637]:
from sklearn.ensemble import AdaBoostClassifier

In [638]:
'''import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import f1_score


# Train the XGBoost model
model = xgb.XGBClassifier(learning_rate=1.75, n_estimators=290)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

f1 = f1_score(y_test, y_pred, average='macro')

print("F1 Score:", f1)'''

'import xgboost as xgb\nimport lightgbm as lgb\nfrom sklearn.metrics import f1_score\n\n\n# Train the XGBoost model\nmodel = xgb.XGBClassifier(learning_rate=1.75, n_estimators=290)\nmodel.fit(X_train, y_train)\n\n# Make predictions on the test set\ny_pred = model.predict(X_test)\n\nf1 = f1_score(y_test, y_pred, average=\'macro\')\n\nprint("F1 Score:", f1)'

In [639]:
import lightgbm as lgb
from sklearn.metrics import f1_score

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_boost_round': 300,
    'learning_rate': 1,
    'max_depth': 4
}

model = lgb.train(params, train_data)
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.4).astype(int)  # Convert probabilities to binary predictions
f1 = f1_score(y_test, y_pred, average='macro')
print("F1 Score:", f1)

/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Number of positive: 3666, number of negative: 13272
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003357 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3405
[LightGBM] [Info] Number of data points in the train set: 16938, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.216436 -> initscore=-1.286555
[LightGBM] [Info] Start training from score -1.286555
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

In [640]:
Y_pred=model.predict(test)
Y_pred = (Y_pred > 0.4).astype(int)  # Convert probabilities to binary predictions
result=pd.DataFrame(Y_pred, columns=['pred'])
result.to_csv('lgb.csv', index=False)